In [ ]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import re, string, random
import math
import random
from collections import defaultdict
from pprint import pprint
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string


import numpy as np
import pandas as pd

In [ ]:
import pandas as pd

reviews = pd.read_csv(r"C:\Users\utkar\Desktop\Research Paper\amazon_reviews_sample.csv")
df = pd.DataFrame(reviews)
df.tail()

<strong><h1>THIS IS THE FINAL NEGATION FUNCTION</h1></strong>

In [ ]:
#sentence = word_tokenize("THis is not good and not bad also but not good also")
def Negation(sentence):
    #sentence = tokens[2]
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't","non-"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                for l in syn.lemmas():
                    #synonyms.append(l.name())
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                        #sentence.pop(i-1) # If the antonym of the word ahead negation exists then pop the negation and replace the word ahead with antonym
                max_dissimilarity = 0     #If antonym does not exist then leave it alone.
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                
                    temp = 1 - word1.wup_similarity(word2) #Replacing with the antonym that has the maximum level of dissimilarity as per the wordnet in nltk
                    if temp>max_dissimilarity:
                        #print(temp)
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    #print(sentence)
    while '' in sentence:
        sentence.remove('')
    return sentence

#print(Negation(sentence))

In [ ]:
documents = ["This is not as good as that vial", "THis is not good", "This will actually do. It is not so bad", "I don't like to go swimming"]
tokens = [word_tokenize(sentences) for sentences in documents]
negated_tokens = [Negation(Lists) for Lists in tokens]
print(negated_tokens)
print(Negation(word_tokenize("I don't like this place")))

In [ ]:
tokens = [word_tokenize(sentence) for sentence in df.review]
negated_tokens = [Negation(Lists) for Lists in tokens]
print(negated_tokens[:100])

In [ ]:
sentences = [' '.join(r) for r in negated_tokens]
df['Negated']=sentences
df.tail(20)

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    #doco_clean = ([i for i in doco_clean if i not in stop])
    doco_clean = ([i for i in doco_clean if i.isalpha()]) # removing numeric
    #doco_clean = ([[word for word in item if word.isalpha()] for item in doco_clean])
#     doco_clean = [spell(word) for word in doco_clean]
#     p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
#     doco_clean = ([spell(k) for k in doco_clean])
    return doco_clean

In [ ]:
print(clean_document("I don't like this place"))

In [ ]:
review_clean = [clean_document(doc) for doc in df['Negated']]
sentences = [' '.join(r) for r in review_clean]
df['cleantext']=sentences
df.tail(20)

In [ ]:
vect = CountVectorizer(max_features = 5000)
vect.fit(df.cleantext)
vect_bow = vect.transform(df.cleantext)
vect_df = pd.DataFrame(vect_bow.toarray(), columns = vect.get_feature_names())
vect_df.head()

In [ ]:
y = df.score
X_bow = vect_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#So BOW-unigrams, preprocessing, 5000 features -> 84.25%
#After new Negation Function -> 85.2%

In [ ]:
#for tfidf
vect1 = TfidfVectorizer(max_features = 5000)
vect1.fit(df.cleantext)
vect1_tfidf = vect1.transform(df.cleantext)
vect1_df = pd.DataFrame(vect1_tfidf.toarray(), columns = vect1.get_feature_names())
vect1_df.head()

In [ ]:
y = df.score
X_tfidf = vect1_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#So TFIDF-unigrams, preprocessing, 5000 features -> 85.35%
#After new Negation function -> 87.3%

In [ ]:
#Testing for BOW, uni+bigrams
vect = CountVectorizer(max_features = 10000, ngram_range=(1,2))
vect.fit(df.cleantext)
vect_bow = vect.transform(df.cleantext)
vect_df = pd.DataFrame(vect_bow.toarray(), columns = vect.get_feature_names())
vect_df.head()

In [ ]:
y = df.score
X_bow = vect_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#So BOW-unigrams+bigrams, preprocessing, 5000 features -> 85.15%
#So upon doubling the no. of features the accuracy -> 85.2%. Accuracy increases exactly by .05% in both TFIDF and BOW
#After new Negation function -> 87.3

In [ ]:
#Testing for BOW only bigrams
vect = CountVectorizer(max_features = 5000, ngram_range=(2,2))
vect.fit(df.cleantext)
vect_bow = vect.transform(df.cleantext)
vect_df = pd.DataFrame(vect_bow.toarray(), columns = vect.get_feature_names())
vect_df.head()

In [ ]:
y = df.score
X_bow = vect_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#So BOW-only bigrams, preprocessing, 5000 features -> 76.7% ; The accuracy drops from uni+bigrams
#After new Negation Function -> 81.4

In [ ]:
#TFIDF testing for uni+bi
vect1 = TfidfVectorizer(max_features = 10000, ngram_range = (1,2))
vect1.fit(df.cleantext)
vect1_tfidf = vect1.transform(df.cleantext)
vect1_df = pd.DataFrame(vect1_tfidf.toarray(), columns = vect1.get_feature_names())
vect1_df.head()

In [ ]:
y = df.score
X_tfidf = vect1_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#So TFIDF-uni+bi, preprocessing, 5000 features -> 86.35%
#In 10000 features the accuracy -> 86.4%. So upon doubling the no. of features there isnt considerable change in accuracy.
#Accuracy increases exactly by .05% in both TFIDF and BOW

In [ ]:
#Testing for TFIDF only bi
vect1 = TfidfVectorizer(max_features = 5000, ngram_range = (2,2))
vect1.fit(df.cleantext)
vect1_tfidf = vect1.transform(df.cleantext)
vect1_df = pd.DataFrame(vect1_tfidf.toarray(), columns = vect1.get_feature_names())
vect1_df.head()

In [ ]:
y = df.score
X_tfidf = vect1_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#So TFIDF-only bi, preprocessing, 5000 features -> 77.15% accuracy drops a lot from uni+bi